# TP MODULO 2 - PROGRAMACION EN ANALISIS DE DATOS

En esta sección se podrá:


1.   Importar Librerías
2.   Leer dataset
3.   Renombrar columna
4.   Conectar a la base de datos
5.   Insertar datos a las tablas






In [ ]:
# 1 Importar librerías

import pandas as pd
import sqlite3
import plotly.express as px



# 2 Leer dataset

csv_url = "https://docs.google.com/spreadsheets/d/16PYjlbnrQjjgJs21tPym98-m-noSYjNn28QZ844dyxs/export?format=csv"

try:
    df = pd.read_csv(csv_url)

    # 3. Renombrar columna Country -> Pais

    if "Country" in df.columns:
        df.rename(columns={"Country": "Pais"}, inplace=True)
    else:
        print("Column 'Country' not found in DataFrame.")

    # Muestra el encabezado del DataFrame para verificar si la carga fue exitosa

    display(df.head())

    # 4. Conectar / crear base SQLite

    # Se utiliza una carpeta local "dataset_2020_2025.db"

    # Conexión a la base de datos SQLite

    sqlite_db_path = "dataset_2020_2025.db"
    conn = sqlite3.connect(sqlite_db_path)
    cursor = conn.cursor()

    # 5. Insertar datos
    # sobrescribirá la tabla si existe.

    df.to_sql("data_2020_2025_wide", conn, if_exists="replace", index=False)

    conn.close()

except Exception as e:
    print(f"An error occurred: {e}")

,Pais,2020,2021,2022,2023,2024,2025
0,Afghanistan,20136,14278.0,14501.0,17248.0,NaN,NaN
1,Albania,15271,18086.0,19185.0,23388.0,27259.0,28372.0
2,Algeria,164774,185850.0,225709.0,247789.0,264913.0,268885.0
3,Andorra,2885,3325.0,3376.0,3786.0,4038.0,4035.0
4,Angola,66521,84375.0,142442.0,109764.0,115946.0,113343.0


Este notebook se conecta a la base de datos SQLite, carga los datos en formato **long**
y genera un gráfico interactivo de la evolución por país utilizando **Plotly Express**.




In [26]:
sqlite_db_path = "dataset_2020_2025.db"
conn = sqlite3.connect(sqlite_db_path)

# Leer tabla long
df_long = pd.read_sql("SELECT * FROM data_2020_2025_long;", conn)
conn.close()

df_long.head()

,Pais,Anio,Valor
0,Afghanistan,2020,20136.0
1,Albania,2020,15271.0
2,Algeria,2020,164774.0
3,Andorra,2020,2885.0
4,Angola,2020,66521.0


# Kpis

En este bloque:

* KPI 1: muestra la suma global de valores para el año seleccionado.
* KPI 2: muestra el país líder en ese año y su valor.

In [27]:
import plotly.graph_objects as go

# Crear figura con 2 indicadores
fig_kpi_dropdown = go.Figure()

# Años disponibles
anios = sorted(df_long["Anio"].unique())

# --- Valores iniciales (primer año)
anio_inicial = anios[0]
df_anio = df_long[df_long["Anio"] == anio_inicial]
suma_global = df_anio["Valor"].sum()
pais_top = df_anio.loc[df_anio["Valor"].idxmax(), "Pais"]
valor_top = df_anio["Valor"].max()

fig_kpi_dropdown.add_trace(go.Indicator(
    mode="number",
    value=suma_global,
    title={"text": f"Suma Global {anio_inicial}"},
    domain={"row": 0, "column": 0}

))

fig_kpi_dropdown.add_trace(go.Indicator(
    mode="number",
    value=valor_top,
    title={"text": f"Top País {anio_inicial}: {pais_top}"},
    domain={"row": 0, "column": 1}

))

# --- Crear frames: uno por año
frames = []
for anio in anios:
    df_anio = df_long[df_long["Anio"] == anio]
    suma_global = df_anio["Valor"].sum()
    pais_top = df_anio.loc[df_anio["Valor"].idxmax(), "Pais"]
    valor_top = df_anio["Valor"].max()

    frames.append(go.Frame(
        data=[
            go.Indicator(
                mode="number",
                value=suma_global,
                title={"text": f"Suma Global {anio}"},
                domain={"row": 0, "column": 0}

            ),
            go.Indicator(
                mode="number",
                value=valor_top,
                title={"text": f"Top País {anio}: {pais_top}"},
                domain={"row": 0, "column": 1}

            )
        ],
        name=str(anio)
    ))

fig_kpi_dropdown.frames = frames

# --- Dropdown con botones para los frames
buttons = [
    dict(
        label=str(anio),
        method="animate",
        args=[[str(anio)], {"mode": "immediate", "frame": {"duration": 0, "redraw": True}}]
    )
    for anio in anios
]

fig_kpi_dropdown.update_layout(
    grid={"rows": 1, "columns": 2},
    updatemenus=[{
        "buttons": buttons,
        "direction": "down",
        "x": 0.5,
        "xanchor": "center",
        "y": 1.2,
        "yanchor": "top"
    }],
    height=250,
    margin={"t":60,"b":0}
)

fig_kpi_dropdown.show()


# Gráficos:
Comenzamos con las siguientes visualizaciones de gráficos:


**Gráfico de barras comparativo por año con dropdown por año**

In [28]:
import plotly.express as px
import plotly.graph_objects as go

# Crear un gráfico de barras para cada año
fig_bar_dropdown = px.bar(
    df_long[df_long["Anio"]==2020].sort_values("Valor", ascending=False).head(15),
    x="Pais",
    y="Valor",
    text_auto=".2s",
    title="Top 15 países - Año 2020"
)

# Crear una lista de frames (uno por cada año)
frames = []
buttons = []

for anio in sorted(df_long["Anio"].unique()):
    df_anio = df_long[df_long["Anio"]==anio].sort_values("Valor", ascending=False).head(15)
    frames.append(
        go.Bar(x=df_anio["Pais"], y=df_anio["Valor"], text=df_anio["Valor"])
    )
    buttons.append(
        dict(
            label=str(anio),
            method="update",
            args=[
                {"x": [df_anio["Pais"]], "y": [df_anio["Valor"]], "text": [df_anio["Valor"]]},
                {"title": f"Top 15 países - Año {anio}"}
            ]
        )
    )

# Agregar dropdown
fig_bar_dropdown.update_layout(
    updatemenus=[{
        "buttons": buttons,
        "direction": "down",
        "x": 0.0,
        "xanchor": "left",
        "y": 2.2,
        "yanchor": "top"
    }]
)

fig_bar_dropdown.update_layout(xaxis_tickangle=45)
fig_bar_dropdown.show()





**Gráfico de líneas con dropdown de país**

In [29]:
# Lista de países únicos
paises = df_long["Pais"].unique()

# Gráfico inicial con todos los países
fig_line = px.line(
    df_long,
    x="Anio",
    y="Valor",
    color="Pais",
    title="Evolución de valores por País (2020-2025)",
)

# Crear dropdown: un botón por país + botón para ver todos
buttons = [
    dict(
        label=p,
        method="update",
        args=[
            {"visible": [p == pais for pais in df_long["Pais"]]},
            {"title": f"Evolución de {p}"}
        ]
    )
    for p in paises
]

# Agregar opción para ver todos
buttons.insert(0, dict(
    label="Todos los países",
    method="update",
    args=[{"visible": [True] * len(df_long)}, {"title": "Evolución de todos los países"}]
))

fig_line.update_layout(
    updatemenus=[{"buttons": buttons, "direction": "down", "x": 0.0, "y": 2.10}],
    legend_title="País"
)

fig_line.show()

**Mapa mundial (Choropleth) animado**

In [30]:
fig_map_slider = px.choropleth(
    df_long,
    locations="Pais",
    locationmode="country names",
    color="Valor",
    hover_name="Pais",
    animation_frame="Anio",   # <- clave para el slider
    color_continuous_scale="Blues",
    title="Mapa mundial - Evolución 2020-2025",
)
fig_map_slider.update_layout(
    geo=dict(showframe=False, showcoastlines=True, projection_type="equirectangular"),
    coloraxis_colorbar=dict(title="Valor")
)
fig_map_slider.show()


# Exportación del Dashboard Integral

Se le aplico estilo CSS

In [35]:
import plotly.io as pio

# Exportar cada gráfico como bloque HTML
html_kpi = pio.to_html(fig_kpi_dropdown, full_html=False, include_plotlyjs='cdn')
html_line = pio.to_html(fig_line, full_html=False, include_plotlyjs=False)
html_bar = pio.to_html(fig_bar_dropdown, full_html=False, include_plotlyjs=False)
html_map = pio.to_html(fig_map_slider, full_html=False, include_plotlyjs=False)

# Combinar con estilo CSS
with open("dashboard_integral.html", "w", encoding="utf-8") as f:
    f.write("""
    <html>
    <head>
        <title>Dashboard Integral - TP Segundo Módulo</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 20px;
                background-color: #f9f9f9;
            }
            h1 {
                text-align: center;
                color: #2c3e50;
                margin-bottom: 40px;
            }
            h2 {
                color: #34495e;
                margin-top: 40px;
                margin-bottom: 10px;
                text-align: center;
            }
            .section {
                background: white;
                border-radius: 12px;
                box-shadow: 0 4px 8px rgba(0,0,0,0.1);
                padding: 20px;
                margin-bottom: 40px;
            }
        </style>
    </head>
    <body>
        <h1>Dashboard Integral - TP Segundo Módulo</h1>

        <div class="section">
            <h2>KPIs Globales</h2>
            """ + html_kpi + """
        </div>

        <div class="section">
            <h2>Evolución por País</h2>
            """ + html_line + """
        </div>

        <div class="section">
            <h2>Top 15 Países por Año</h2>
            """ + html_bar + """
        </div>

        <div class="section">
            <h2>Mapa Mundial 2020-2025</h2>
            """ + html_map + """
        </div>

    </body>
    </html>
    """)

print("Dashboard integral con estilo generado en 'dashboard_integral.html'")

Dashboard integral con estilo generado en 'dashboard_integral.html'
